# Recurring runs with the KFP SDK

If you're running on a local cluster, expose the GUI and API, respectively, with
the following commands:

```
kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80
kubectl port-forward -n kubeflow svc/ml-pipeline-ui 3000:80
```

The rest of this demo assumes that you're running locally.

Instantiate the KFP SDK client. Set the host variable to the url and port where
you expose the KFP API. 

In [ ]:
import kfp

host = 'http://localhost:3000'
client = kfp.Client(host=host)

Create a pipeline component from the provided component file. This component
retrieves and executes a script from a provided URL.

In [ ]:
run_script = kfp.components.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/examples/master/demos/recurring/component.yaml'
)

Create a pipeline function.

In [ ]:
def pipeline(url):
    run_script_task = run_script(url=url)

Compile the pipeline function. We will pass the resulting yaml to the pipeline
execution invocations.

In [ ]:
kfp.compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='download.yaml',
)

Create a parameters dictionary with the url key. 

In [ ]:
parameters = {
    'url': 'https://raw.githubusercontent.com/kubeflow/examples/master/demos/recurring/success.sh'
}

We can _optionally_ validate the pipeline with a single run before creating a recurring run.

In [ ]:
result = client.create_run_from_pipeline_func(
    pipeline_func=pipeline,
    arguments=parameters,
)

We can retrieve the result of the pipeline run through the Kubeflow GUI, which
is the recommended approach. That being said, we can also interrogate the result
programmatically.

In [ ]:

result.wait_for_run_completion()
print(result.run_info)

Now that we've validated a single run, let's create a recurring run.

We first need to create an experiment since the `create_recurring_run` method
requires an `experiment_id`.

In [ ]:
experiment = client.create_experiment('test')

job = client.create_recurring_run(
    experiment_id=experiment.id,
    job_name='test',
    cron_expression='*/2 * * * *',  # Runs once every two minutes.
    pipeline_package_path='download.yaml',  # Pass in compiled output.
    params=parameters,
)

The Kubeflow Pipelines GUI provides an excellent interface for interacting with
recurring runs, but you can interrogate the job programmatically if you prefer.

In [ ]:
print(job)

In the GUI, you can retrieve the logs of an individual run. They should
culminate with `Success!`.

To disable the recurring run:

In [ ]:
client.disable_job(job.id)

To list recurring runs:

In [ ]:
client.list_recurring_runs()

To get details about an individual recurring run:

In [ ]:
client.get_recurring_run(job.id)

To delete a recurring run programmatically:

In [ ]:
result = client.delete_job(job.id)

Additional recurring run interactions via the SDK are documented [here](https://kubeflow-pipelines.readthedocs.io/en/stable/).